In [7]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [10]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [11]:
# import PPIGraph

# Read per cell data

In [13]:
# Read PPI dfs
PPI_save_path =  Path(r'Y:\coskun-lab\Thomas\15_PLA\data\OCT Cell Culture\Whole\PPI')

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,9655.000000,9655.000000,9655.000000,9655.000000
mean,332.755256,18.820404,8.390989,10.429415
std,180.615431,29.040642,10.916339,21.016527
min,2.000000,1.000000,0.000000,-7.000000
25%,176.000000,4.000000,2.000000,2.000000
50%,326.000000,11.000000,5.000000,5.000000
75%,475.000000,22.000000,11.000000,11.000000
max,778.000000,874.000000,202.000000,774.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_38084\3581057356.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,2081.000000,2081.000000,2081.000000
mean,87.319077,38.930802,48.388275
std,78.104479,29.733528,57.558508
min,1.000000,0.000000,-13.000000
25%,41.000000,21.000000,17.000000
50%,68.000000,33.000000,33.000000
75%,112.000000,51.000000,61.000000
max,997.000000,336.000000,842.000000


In [14]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
# df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cyto'] >= 0).all())

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,7129.000000,7129.000000,7129.000000,7129.000000
mean,339.626876,14.332866,7.021321,7.311544
std,181.049865,13.471075,6.836728,8.711061
min,20.000000,1.000000,0.000000,-5.000000
25%,185.000000,5.000000,2.000000,2.000000
50%,329.000000,10.000000,5.000000,4.000000
75%,481.000000,19.000000,10.000000,10.000000
max,752.000000,69.000000,46.000000,64.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_38084\2711417620.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,1510.000000,1510.000000,1510.000000
mean,67.668212,33.149007,34.519205
std,31.996032,16.639663,24.150085
min,21.000000,0.000000,1.000000
25%,43.000000,22.000000,17.000000
50%,63.000000,31.000000,29.000000
75%,87.000000,43.000000,45.000000
max,292.000000,143.000000,246.000000


# ML model

In [15]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import wandb

In [16]:
scaler = StandardScaler()

In [17]:
condition = 'ML'
project_name = f'PLA_5PPI_batch'

In [18]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [19]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']
df_count_cell = pd.DataFrame(scaler.fit_transform(df_count_cell), index=df_count_cell.index, columns=df_count_cell.columns, )

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [20]:
df_count_cell

Cell                                         \
PPI                CylinE & CDK2 Mcl-1 & BAK P-ERK & c-MYC TEAD1 & YAP1   
Condition  FOV Id                                                         
HCC827Ctrl FW1 52      -0.984185   -0.372357     -0.034931     0.002731   
               53       0.556125   -0.544249      0.970306    -0.924964   
               54       1.396295   -0.372357      1.206832     0.002731   
               55      -0.144016   -0.544249     -0.921904    -0.821887   
               56      -0.704129   -0.200465      1.265964     0.002731   
...                          ...         ...           ...          ...   
HCC827Osim FW2 519     -0.844157   -0.544249     -0.567115     0.002731   
               520     -0.564100   -0.028573      0.556385    -0.409578   
               523     -1.404270   -0.200465     -1.099299    -1.234196   
               524     -1.264241   -0.028573     -0.034931    -1.646504   
               526     -1.124213   -0.200465     -0.271457     0.105808   

                                 
PPI                p-AKT & mTOR  
Condition  FOV Id                
HCC827Ctrl FW1 52     -0.295713  
               53     -0.295713  
               54     -0.450595  
               55     -0.605478  
               56      0.478700  
...                         ...  
HCC827Osim FW2 519     0.633582  
               520    -0.915243  
               523    -0.450595  
               524     0.323817  
               526    -0.605478  

[1510 rows x 5 columns]

In [22]:
## Train on P10 an'P10d evaluate on P21
train = 'FW1'
held = 'FW2'

# Run model on cell count
df_train = df_count_cell[df_count.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count.index.get_level_values('FOV') == held]
X = df_train.values
y = le.transform(df_train.index.get_level_values(0))
X_held = df_held.values
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))
print(np.unique(y, return_counts=True), np.unique(y_held, return_counts=True))

832 832 678 678
(array([0, 1]), array([522, 310], dtype=int64)) (array([0, 1]), array([428, 250], dtype=int64))


In [23]:
## Train on P10 an'P10d evaluate on P21
train = 'FW1'
held = 'FW2'

# Run model on cell count
df_train = df_count_cell[df_count_cell.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count_cell.index.get_level_values('FOV') == held]

X = df_train.values
# X = scaler.fit_transform(X)
y = le.transform(df_train.index.get_level_values(0))

X_held = df_held.values
# X_held = scaler.fit_transform(X_held)
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

# K fold training
kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)
        y_pred_held = model.predict(X_held)
        y_probas_held = model.predict_proba(X_held)

        run = wandb.init(project=project_name, group=model_name+'_'+train, name=model_name+f'_{train}_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        accuracy_held = metrics.accuracy_score(y_held, y_pred_held)
        f1 = metrics.f1_score(y_test, y_pred)
        f1_held = metrics.f1_score(y_held, y_pred_held)
        # auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        # auc_held = metrics.roc_auc_score(y_held, y_probas_held[:, 1])
        auc = metrics.roc_auc_score(y_test, y_pred)
        auc_held = metrics.roc_auc_score(y_held, y_pred_held)
        wandb.log({"accuracy": accuracy, 'accuracy_held': accuracy_held, 'f1':f1, 'f1_held': f1_held, 'auc': auc, 'auc_held': auc_held})
    run.finish()

832 832 678 678


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.76259
accuracy_held,0.69764
auc,0.72096
auc_held,0.68649
f1,0.625


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.68953
accuracy_held,0.71681
auc,0.669
auc_held,0.7025
f1,0.59434


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.72924
accuracy_held,0.71091
auc,0.70778
auc_held,0.6945
f1,0.63415


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.76978
accuracy_held,0.74336
auc,0.74007
auc_held,0.74183
f1,0.65217


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.71119
accuracy_held,0.72714
auc,0.68867
auc_held,0.71983
f1,0.61538


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.77978
accuracy_held,0.72566
auc,0.7559
auc_held,0.7145
f1,0.69347


In [24]:
## Train on P10 an'P10d evaluate on P21
train = 'FW2'
held = 'FW1'

# Run model on cell count
df_train = df_count_cell[df_count_cell.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count_cell.index.get_level_values('FOV') == held]

X = df_train.values
# X = scaler.fit_transform(X)
y = le.transform(df_train.index.get_level_values(0))

X_held = df_held.values
# X_held = scaler.fit_transform(X_held)
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

# K fold training
kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)
        y_pred_held = model.predict(X_held)
        y_probas_held = model.predict_proba(X_held)

        run = wandb.init(project=project_name, group=model_name+'_'+train, name=model_name+f'_{train}_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        accuracy_held = metrics.accuracy_score(y_held, y_pred_held)
        f1 = metrics.f1_score(y_test, y_pred)
        f1_held = metrics.f1_score(y_held, y_pred_held)
        # auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        # auc_held = metrics.roc_auc_score(y_held, y_probas_held[:, 1])
        auc = metrics.roc_auc_score(y_test, y_pred)
        auc_held = metrics.roc_auc_score(y_held, y_pred_held)
        wandb.log({"accuracy": accuracy, 'accuracy_held': accuracy_held, 'f1':f1, 'f1_held': f1_held, 'auc': auc, 'auc_held': auc_held})
    run.finish()

678 678 832 832


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.69027
accuracy_held,0.73197
auc,0.66533
auc_held,0.67504
f1,0.57831


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.75221
accuracy_held,0.73197
auc,0.72988
auc_held,0.67897
f1,0.65854


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.74336
accuracy_held,0.69952
auc,0.69374
auc_held,0.63608
f1,0.59155


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.69469
accuracy_held,0.70433
auc,0.67121
auc_held,0.6648
f1,0.58683


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.74779
accuracy_held,0.73558
auc,0.71143
auc_held,0.67791
f1,0.62745


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.81416
accuracy_held,0.74399
auc,0.78161
auc_held,0.68527
f1,0.72368


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.76549
accuracy_held,0.73197
auc,0.72461
auc_held,0.67439
f1,0.63946
